In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.layers import *
from local.data.pipeline import *
from local.data.source import *
from local.data.core import *
from local.optimizer import *
from local.learner import *
from torch.utils.data import TensorDataset

In [ ]:
# default_exp utils.test

# Synthetic Learner

> For quick testing of the training loop and Callbacks

In [ ]:
#export
from torch.utils.data import TensorDataset

def synth_data(a=2, b=3, bs=16, n_train=10, n_valid=2, cuda=False):
    def get_data(n):
        x = torch.randn(bs*n, 1)
        return TensorDataset(x, a*x + b + 0.1*torch.randn(bs*n, 1))
    train_ds = get_data(n_train)
    valid_ds = get_data(n_valid)
    tfms = Cuda() if cuda else None
    train_dl = TfmdDL(train_ds, bs=bs, shuffle=True, after_batch=tfms, num_workers=0)
    valid_dl = TfmdDL(valid_ds, bs=bs, after_batch=tfms, num_workers=0)
    return DataBunch(train_dl, valid_dl)

In [ ]:
#export
class RegModel(Module):
    def __init__(self): self.a,self.b = nn.Parameter(torch.randn(1)),nn.Parameter(torch.randn(1))
    def forward(self, x): return x*self.a + self.b

In [ ]:
# export
def synth_learner(n_trn=10, n_val=2, cuda=False, **kwargs):
    return Learner(RegModel(), synth_data(n_train=n_trn,n_valid=n_val, cuda=cuda), MSELossFlat(), 
                   opt_func=partial(SGD, mom=0.9), **kwargs)

In [ ]:
#hide
from local.notebook.export import *
notebook2script(all_fs=True)